In [1]:
# AutoEncoders

# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
# Importing the dataset
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')



In [3]:
# Preparing the training set and the test set
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')#user_id,movie_id,ratings
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')
test_set
training_set

array([[        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       [        1,         4,         3, 876893119],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

In [4]:
test_set.shape
training_set.shape

(79999, 4)

In [5]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [6]:
# Converting the data into an array with users in lines and movies in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)


In [7]:
# Creating the architecture of the Neural Network
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)


In [8]:
# # Training the SAE
# nb_epoch = 200
# for epoch in range(1, nb_epoch + 1):
#     train_loss = 0
#     s = 0.
#     for id_user in range(nb_users):
#         input = Variable(training_set[id_user]).unsqueeze(0)
#         target = input.clone()
#         if torch.sum(target.data > 0) > 0:
#             output = sae(input)
#             target = Variable(test_set[id_user]).unsqueeze(0)
#             output[target == 0] = 0
#             loss = criterion(output, target)
#             mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
#             loss.backward()
#             train_loss += np.sqrt(loss.data*mean_corrector)
#             s += 1.
#             optimizer.step()
#     print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))


In [9]:
# # Testing the SAE
# test_loss = 0
# s = 0.
# for id_user in range(nb_users):
#     input = Variable(training_set[id_user]).unsqueeze(0)
#     #target = Variable(test_set[id_user])
#     target = Variable(test_set[id_user]).unsqueeze(0)
#     if torch.sum(target.data > 0) > 0:
#         output = sae(input)
#         target.require_grad = False
#         output[(target == 0).unsqueeze(0)] = 0
#         loss = criterion(output, target)
#         mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
#         test_loss += np.sqrt(loss.data*mean_corrector)
#         s += 1.
# print('test loss: '+str(test_loss/s))

In [10]:
# Training the SAE
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()

            train_loss += np.sqrt(loss.item()*mean_corrector)

            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))


epoch: 1 loss: 1.7721755118400813
epoch: 2 loss: 1.0967002187920258
epoch: 3 loss: 1.0534095459116226
epoch: 4 loss: 1.0385055352103987
epoch: 5 loss: 1.0308675221909651
epoch: 6 loss: 1.0267049404366253
epoch: 7 loss: 1.023661896311819
epoch: 8 loss: 1.0219707438837125
epoch: 9 loss: 1.020642215823833
epoch: 10 loss: 1.0197947756968995
epoch: 11 loss: 1.018717435740475
epoch: 12 loss: 1.0183952987463076
epoch: 13 loss: 1.0176740063565333
epoch: 14 loss: 1.0175371352373956
epoch: 15 loss: 1.0172999306266928
epoch: 16 loss: 1.0169839191889876
epoch: 17 loss: 1.0168697234790554
epoch: 18 loss: 1.0164771307296556
epoch: 19 loss: 1.016428791639911
epoch: 20 loss: 1.015862903922607
epoch: 21 loss: 1.016080619715819
epoch: 22 loss: 1.0160458584782568
epoch: 23 loss: 1.0158475407878997
epoch: 24 loss: 1.015672981466766
epoch: 25 loss: 1.0155490587595024
epoch: 26 loss: 1.0155520390362334
epoch: 27 loss: 1.0154293108453496
epoch: 28 loss: 1.0148913432486977
epoch: 29 loss: 1.0140114753534288
e

In [ ]:
# # Testing the SAE
# test_loss = 0
# s = 0.
# for id_user in range(nb_users):
#     input = Variable(training_set[id_user]).unsqueeze(0)
#     #target = Variable(test_set[id_user])
#     target = Variable(test_set[id_user]).unsqueeze(0)
#     if torch.sum(target.data > 0) > 0:
#         output = sae(input)
#         target.require_grad = False
#         output[(target == 0).unsqueeze(0)] = 0
#         loss = criterion(output, target)
#         mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
#         test_loss += np.sqrt(loss.data[0]*mean_corrector)
#         s += 1.
# print('test loss: '+str(test_loss/s))

In [ ]:
# Testing the SAE
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)


    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        #test_loss += np.sqrt(loss.data[0]*mean_corrector)
        test_loss += np.sqrt(loss.item()*mean_corrector)


        s += 1.
print('test loss: '+str(test_loss/s))

In [ ]:
for i in range(1,1682):
    print(target[0][i])

In [ ]:
target.shape

In [ ]:
user_id = 0
movie_title = movies.iloc[:nb_movies, 1:2]
user_rating = training_set.data.numpy()[user_id, :].reshape(-1,1)
user_target = test_set.data.numpy()[user_id, :].reshape(-1,1)
 
user_input = Variable(training_set[user_id]).unsqueeze(0)
predicted = sae(user_input)
predicted = predicted.data.numpy().reshape(-1,1)
 
# Join all info in one dataset
result_array = np.hstack([movie_title, user_target, predicted])
result_array = result_array[result_array[:, 1] > 0]
result_df = pd.DataFrame(data=result_array, columns=['Movie', 'Target Rating', 'Predicted'])

In [ ]:
result_df
t#raining_set.data.numpy()[user_id, :]

In [ ]:
training_set

In [ ]:
user_rating

In [ ]:
user_id = 0
movie_title = movies.iloc[:nb_movies, 1:2]
user_rating = training_set.data.numpy()[user_id, :].reshape(-1,1)
user_target = test_set.data.numpy()[user_id, :].reshape(-1,1)
 
user_input = Variable(training_set[user_id]).unsqueeze(0)
predicted = sae(user_input)
predicted = predicted.data.numpy().reshape(-1,1)
 
# Join all info in one dataset
result_array = np.hstack([movie_title, user_target, predicted])
result_array = result_array[result_array[:, 1] > 0]
result_df = pd.DataFrame(data=result_array, columns=['Movie', 'Target Rating', 'Predicted'])

In [ ]:
user_rating